In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import string
import pickle as pkl
import numpy as np

TRUE_CUTOFF = 2000

In [2]:
replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
red_flag_char = [" AVE ", " ST ", " AV ", " BLVD ", " BLV ", " RD ", " DR "]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

In [3]:
# remake the dict
map_dict = {}

# use previous dict
# map_dict = pkl.load(open('trueheaders_match_dict.pkl', 'rb'))

In [4]:
def clean_header(h):
    red_flag = False
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = ''.join(hl).upper()
    for rf in red_flag_char:
        if rf in h: red_flag = True
    if cnt > 4 or red_flag: h = ""
    return h
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the ocr header and the true header
    return fuzz.ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
            score_tuple = sorted_score[0]
            if score_tuple[0] > 90:
                print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = score_tuple
            else:
                print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], "no_header")
    return map_dict

In [5]:
raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

In [6]:
df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True).sort_values("count")
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df[TRUE_CUTOFF:].true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

LOG: Not matched, BLECTRICAL CONSTRUCTIO and ROAUL CONSTRUCTIO with score 77
LOG: Not matched, HAIR GAODS -MFRS and HAIR GOODS (HUMA DEALERS with score 65
LOG: Not matched, SOAPS-INDUSTRIAL AND LAUNDRY and INDUSTRIAL HARDWARE with score 64
LOG: Not matched, AUTOMOBILE TOWIHG SERVICE and AUTOMOBILE PARTS AND SERV- ICE with score 73
LOG: Not matched, GLUE MFRS and N E'EDLE MFRS with score 64
LOG: Matched! TYPEWRITER DEALERS and FNYPEWRITER DEALERS with score 92
LOG: Not matched, JEWELRY REPAIRING and JEWE LRY-RETAIL with score 69
LOG: Matched! REAL ESTATE  INSURANCE and LF'RE'AL ESTATE  INSURANCE with score 92
LOG: Not matched, DYERS and BERS with score 67
LOG: Not matched, TRACTOR TRAILERS- HEAVY TRUCKS and C-ONTRACTORS-TRAILER SERVICE with score 62
LOG: Matched! CHINA, CROCKERY, GLASSWARE AND EAR THENWARE-WHOL E- SALE and CHINA, CROCKERY, GLASSWARE AND EARTHE NWARE-WHOLE- SALE with score 97
LOG: Matched! MEN'S PUNISHING GOODS and MEN'S FURNISHING GOODS with score 93
LOG: Matched! WELDI

LOG: Not matched, MAILING-LETTERS-ADVERTLSLNA and  MAILLNG-LETTERS-ADVERTISING with score 84
LOG: Not matched, RUQS and BERS with score 50
LOG: Not matched, PACIGNG-STEAM, WATER AND AIR and PACKING-STEAM, ETC with score 65
LOG: Not matched, CHINAWARE  GLASSWARE and CHILDREN'S APPAREL with score 53
LOG: Not matched, OORNIOE MANUFACTURERS and COMICE MANUFACTURERS with score 83
LOG: Not matched, ROOFING MATERIALS  ASPHALT A SHINGLES and ASBERSTOS MATERIALS  SHINGLES  with score 66
LOG: Matched! POLDING BOX MANUFACTURERS and POLDLNG BOX MANUFACTURERS with score 96
LOG: Not matched, WEAVING-CLOTHING and MEN'S CLOTHING with score 67
LOG: Not matched, ELECTRLCAL WIRING-INDUSTRLAL and ELECTRIC WIRING with score 65
LOG: Not matched,  OFFICE CLEANERS and HOUSE CLEANERS  with score 71
LOG: Not matched, BLEACHE RS- FABRICS and ELEACHERS-FABRICS with score 89
LOG: Matched! LLNIE, BRICK, CEMENT SAND, HAIR, PLASTER C and LIME, BRICK. CEMENT SAND, HAIR, PLASTER C with score 94
LOG: Matched! ASPHA;LT  

LOG: Matched! CLOTHING MFRS- WOMEN'S AND MISSES and CLOTHING MFRS-WOMEN'S AND MISSES with score 98
LOG: Not matched,  JEWE LRY MANUFAC TUBING and JEWELRY MANUFACTURING with score 89
LOG: Not matched, ENGRAVERS-GOMMERCIAL and I'REAL ESTATE-COMMERCIAL with score 64
LOG: Matched! STAMPING S-ME TAL and BTAMPINGS-ME TAL with score 91
LOG: Not matched, PAVLNG CONTRACTORS and '-'PAVING CONTRACTORS with score 87
LOG: Matched!  ADVE RTISI NG -RADIO BROAD- C ASTING and ADVERTISDIG -RADIO BROAD- CASTING with score 91
LOG: Matched! L DIE MILLING MACHINES and DIE MILLING; MACHINES with score 93
LOG: Not matched,  UNDERWRITERS and  TYPEWRITERS with score 72
LOG: Matched! ELASTIC GOODS MANUFACTURERS and ELASTIC GOODS MANUFAC- TURERS with score 96
LOG: Matched! COAL DEALERS-WHOL AND BROKERS and COAL DEALE RS-WHOLESALE AND BROKERS with score 91
LOG: Not matched, MERCHA'NDISE ON CREDIT and DISPLAYS-MERCHANDISE  COUNTER with score 59
LOG: Not matched, ICE DEALERS-PE'DDLERS SUPPLIED and LHM; DVEALERS-PEDL

LOG: Not matched, TAILORS' TRIMMING and TAILORS-TRIMMINGS with score 88
LOG: Not matched, L'LINVESTMENT SECURITIES and LNVESTMENT ADVISORS with score 65
LOG: Not matched, PENCE DEALERS and FEATHER DEALERS with score 71
LOG: Matched! LIME, BRICK, CEMENT, SAND, SEWER PIPE, PLASTER, ETC  and LIME, BRICK, CEMENT, SAND SEWER PIPE, PLASTER. ETC  with score 97
LOG: Matched! BROKERS-SHIP AND YACHT and BROKERS-SHIP AN D YACHT with score 98
LOG: Not matched,  BOOKBINDERS-C ONTD and BROKERS-INVOSTMENT with score 54
LOG: Not matched, BOMLS-INSURANCE and BONJDS-INSURANCE with score 84
LOG: Not matched, ALES-BREWERS and BRELWERIES with score 64
LOG: Not matched, STEEL FURNITURE AND and OTTICE FURNITURE with score 69
LOG: Matched! SCHOOLS-EVENING and SCHAOLS-EVENING with score 93
LOG: Not matched, RADIO SETST-SALES  SERVICE and OUTBOARD MOTORS-SALES AND SERVICE with score 68
LOG: Not matched, BOARDING  BOOMING HAOUS-ES-  BIOATS-D E-ALERS and BEARINGS-BALL-MFRS  DEALERS with score 53
LOG: Matched! DRU

LOG: Not matched, CHECKLNG ACCOUNTS and LWAVINGS ACCOUNTS with score 71
LOG: Not matched, OHICE PUMITURE and OTTICE FURNITURE with score 73
LOG: Not matched, PACKING-STEAM, WATER and PACKING-STEAM, ETC with score 84
LOG: Matched! MONTRACTORS-PAINTERS and CONTRACTORS-PAINTERS with score 95
LOG: Not matched,  LADDER MFRS and  LADDERS with score 80
LOG: Matched! STAMPS-RUBBER. STEEL AND BRASS and TRAMPS-RUBBER, STEEL AND BRASS with score 93
LOG: Not matched, HATS-MEES  STUDENTS,-RETALL and GLASS DEALERS-RETALL with score 60
LOG: Not matched, BED SPRING  MFRS and ANTOMOBILE SPRING MFRS with score 74
LOG: Not matched, AUTOINOBILE FILLING STATIONS  AUTO FRAME STRAIGHTENING and GASOLINE FILLING STATION EQUIP- MENT with score 60
LOG: Not matched, HEALTH SERVICE and MENTAL HEALTH SERVICES with score 78
LOG: Not matched, HAMMOND SOLOVAX and MONEY TO LOA with score 59
LOG: Not matched, BRIET PRINTERS and BFOOKLET PRINTERS with score 77
LOG: Matched! TEXTILE MIRS-SPECIALTY BLENDS and FTEXTILE MTRS

...

LOG: Matched! VALUABLES TRANSPORTED and VALUABLES TRANSPO-RTED with score 98
LOG: Not matched, CONTRACTORS-HOME IMPROVEMENT and CONTRACTORS' EQUIPMENT with score 72
LOG: Not matched, BANDS  ORCHESTRAS and ASSAYERS  CHEMISTS with score 57
LOG: Not matched, EQUIPMENT and EQUIPMENT-RENTAIS with score 69
LOG: Not matched, BROKERS-FOOD PRODUCTS and HOME CARE PRODUCTS with score 62
LOG: Not matched, SALES AND SERVICE and LAWN SERVICE with score 76
LOG: Not matched, CONTRACTORS-SEWER  DRAINS and  CONTRACTORS-GENERAL with score 71
LOG: Not matched, HOMES AND SANITARIUMS and RHODE ISLAND HOSPITAL TRUST with score 63
LOG: Not matched, ELECTRONIC EQUIPMENT AND SUPPLIES and ELECTRICAL EQUIPMENT AND SUPPLIES- WHOLESALE with score 81
LOG: Not matched, DESLGN and DIES with score 60
LOG: Not matched, ELECTRICAL EQUIPMENT  SUPPLIES and ELECTRICAL EQUIPMENT AND SUPPLIES- WHOLESALE with score 81
LOG: Matched! ACCOUNTANTS-CERTIFIED PUBLIC and ACCAUNTANTS-CERTIFIED PUBLIC with score 96
LOG: Not matched, OR

LOG: Not matched, WATCH  JEWELRY REPAIRERS and LAWN MOWER RE PAIRERS with score 67
LOG: Not matched, REMNANTS-DEALERS and GRANLTE DEALERS with score 71
LOG: Not matched, CHURCHES AND SYNAGOGUES and HOUSE MOVERS AND SHORERS with score 55
LOG: Not matched, GROCERIES- AND MEATS- RETAIL and WOMEWS SUITS AND COATS- RETAIL with score 69
LOG: Not matched, CONFECTIONERY-RETAIL and LCLOTHING DEALERS-RETAIL with score 64
LOG: Not matched, CEMETERIES and CREAMERIES with score 80
LOG: Not matched, TRAVEL AGENCIES and STEAMSHIP AGENCIES with score 67
LOG: Not matched, FUEL OIL DEALERS and FFPUMP DEALERS with score 67
LOG: Not matched, AUTOMOBILE ACCESSORIES AN D PARTS-RETAIL and AUTOMOBILE ACCESORIES AN D PARTS-MFRS with score 88
LOG: Not matched, ADVERTISING COUNSEL  SERVICE and ADVERTISIN G COUNSELNNQ AN D SERVICE with score 88
LOG: Not matched, GONCMTE CONSTRUCTION- CONTRACTORS and GUT STONE CONTRACTORS with score 70
LOG: Not matched, ELECTRICAL EQUIPMENT  SUPPLIES -DEALERS and ELECTRICAL EQUIPM

LOG: Not matched, CLINICS and CANDLCS with score 57
LOG: Not matched, MEATS-WHOLESALE and CHEMIEALS-WHOLESALE with score 82
LOG: Not matched, PUBLIC and MARKEI'S-PUBLIC with score 57
LOG: Not matched, MILL SUPPLIES and MARINE SUPPLIE with score 74
LOG: Not matched, CLOTHING - RETAIL and LCLOTHING DEALERS-RETAIL with score 78
LOG: Not matched, OFFICE SUPPLIES AND EQUIPMENT and PLATERQ SUPPLIES AND EQUIPMENT with score 81
LOG: Not matched, MATTRESS MFRS and ABRASIVES MFRS with score 67
LOG: Not matched, AUTOMOBILE INSURANCE and AUTOMOBILE PINANCINU with score 80
LOG: Not matched, AUTOMOBILE ACCESSORIES AND PARTS-RETAIL and AUTOMOBILE ACCESORIES AN D PARTS-MFRS with score 87
LOG: Not matched, PHYSICIANS AND SURGEONS MD and PHYSICIANS'  SURGEONS' BX- CHANGE with score 71
LOG: Not matched, MILL ENGINEERS-MILL SUPPLIES and OIL BURNER DEALERSL SUPPLIES with score 64
LOG: Not matched, LIQUORS-WHOLESALE and F LORISTS-WHOLESA LE with score 76
LOG: Not matched, BROKERS-MONEY and BROKERS-INVOSTMEN

LOG: Not matched, BUILDING MATERIALS  SUPPLIES and BNILDING MATERIALS  SUPPLIES- USED with score 87
LOG: Matched! ROOFERS and ROOF ERS with score 93
LOG: Not matched, JEWELER'S FINDINGS and JEWELERS  CARDS with score 67
LOG: Not matched, BAKED GOODS-RETAIL and LEATHER GOODS-RETALL with score 74
LOG: Not matched, FUEL AND RANGE OIL and GUIDE AND RIGHT TOP LINES with score 56
LOG: Not matched, CHRISTIAN SCIENCE PRACTITIONERS and INSULATING CFONTRACTORS with score 56
LOG: Not matched, ANTIQUES and PAINTERS with score 63
LOG: Not matched, ELECTRICAL EQUIPMENT AND SUPPLIES and ELECTRICAL EQUIPMENT AND SUPPLIES- WHOLESALE with score 86
LOG: Matched! OIL BURNERS  OIL BURNING EQUIPMENT and OIL BURNERS AND OIL BURNING EQUIPMENT with score 96
LOG: Not matched, PLORISTS-RETAIL and PLASTICS-RETAIL with score 80
LOG: Not matched, MILK DEALERS and BRICK DEALERS with score 80
LOG: Not matched, CATERERS and CREAMERIES with score 67
LOG: Not matched, GROCERS-WHOLESALE and STATIONERS-WHO LESALE with sco

LOG: Not matched, INLAND MARINE and INSURANEE-MARLNE with score 62
LOG: Not matched, MILLINERS and  MILLINERY with score 84
LOG: Not matched, INSURANCE and FLRE INSURANCE with score 78
LOG: Not matched, SERVICE and LAWN SERVICE with score 74
LOG: Not matched, NOVELTIES-MIRS and NOVELTIES-JEWELRY with score 71
LOG: Not matched, NURSERYMEN - NURSES and NURSERY INC with score 60
LOG: Not matched, FINANCE COMPANIES and IINSURANCE COMPANIES with score 86
LOG: Not matched, FURNITURE DEALERS-RETAIL and SAIE DEALERS-RETAIL with score 79
LOG: Not matched, AUTOMOBILE GARAGES and AUTOMOBILE BATTE RIES with score 77
LOG: Not matched, SHEET METAL WORKERS and S'HEET METALS with score 75
LOG: Not matched, AUTOMOBILE DEALERS-USED CARS and AUTOMOBLLES-USED CARS with score 86
LOG: Not matched, CLEANERS AND DYERS and CLEAMERS  DYERS with score 85
LOG: Not matched, PLATERS and PAINTERS with score 80
LOG: Not matched, INVESTMENT SECURITIES and INVESTMEN T BROKERS with score 70
LOG: Not matched, CHURCHES an

In [8]:
pkl.dump(map_dict, open('trueheaders_match_dict.pkl', 'wb'))

In [9]:
def assign_matched(D):
    matched = []
    for h in D.true_headers:
        if h in map_dict: 
            matched.append(map_dict[h][1])
        else: 
            matched.append(h)
    return matched
def assign_score(D):
    scores = []
    for h in D.true_headers:
        if h in map_dict: 
            scores.append(map_dict[h][0])
        else:
            scores.append(np.nan)
    return scores
df = df.assign(matched = assign_matched, score = assign_score).sort_values("count")

In [10]:
known = df.loc[df.matched != "no_header"].reset_index(drop=True).drop_duplicates().sort_values("count")
unknown = df.loc[df.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers").sort_values("count")

In [ ]:
# def internal_match(headers, true_headers, map_dict):
#     # Matches ocr headers to true headers
#     for header in headers:
#         true_headers = [h for h in true_headers if h != header]
#         if header not in map_dict:
#             score_list = [(score(header, true_header), true_header) for true_header in true_headers]
#             sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
#             score_tuple = sorted_score[0]
#             if score_tuple[0] > 90:
#                 print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = score_tuple
#             else:
#                 print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = (score_tuple[0], "no_header")
#     return map_dict
# def assign_matched_internal(D):
#     matched = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             matched.append(internal_dict[h][1])
#         else: 
#             matched.append(h)
#     return matched
# def assign_score_internal(D):
#     scores = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             scores.append(internal_dict[h][0])
#         else:
#             scores.append(np.nan)
#     return scores

# internal_dict = {}
# internal_dict = internal_match(list(unknown.true_headers), list(unknown.true_headers), internal_dict)

# internal_matched = unknown
# internal_matched = internal_matched.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown2 = internal_matched.loc[internal_matched.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# internal_dict = internal_match(list(unknown2.true_headers), list(unknown2.true_headers), internal_dict)

# internal_matched2 = unknown2
# internal_matched2 = internal_matched2.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown3 = internal_matched2.loc[internal_matched2.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# unknown3

In [76]:
def calculate_scores(df):
    l = len(df.true_headers)
    score_matrix = np.zeros((l,l))
    i = 0
    for h in df.true_headers:
        score_matrix[i, :] = df.true_headers.apply(score, args=(h,)).values
        print("Row number: {} of {}".format(i, l - 1))
        i += 1
    return score_matrix

def remove_repeat(df, scores):
    prelist = list(df.true_headers)
    i = 0
    for h in df.true_headers:
        j = 0
        for removal in scores[i, :]:
            if removal and j > i:
                try:
                    df = df[df["true_headers"] != prelist[j]]
                    print("LOG: " + prelist[j] + " deleted!")
                except KeyError:
                    print("LOG: " + prelist[j] + " Already deleted")
            j += 1
        i += 1
    return df

In [53]:
scores = calculate_scores(unknown, 90)
removal_matrix = (scores > 90) & (scores < 100)
internal_matched = remove_repeat(unknown, removal_matrix)
internal_matched = internal_matched.reset_index(drop=True)

Row number: 0 of 9367
Row number: 1 of 9367
Row number: 2 of 9367
Row number: 3 of 9367
Row number: 4 of 9367
Row number: 5 of 9367
Row number: 6 of 9367
Row number: 7 of 9367
Row number: 8 of 9367
Row number: 9 of 9367
Row number: 10 of 9367
Row number: 11 of 9367
Row number: 12 of 9367
Row number: 13 of 9367
Row number: 14 of 9367
Row number: 15 of 9367
Row number: 16 of 9367
Row number: 17 of 9367
Row number: 18 of 9367
Row number: 19 of 9367
Row number: 20 of 9367
Row number: 21 of 9367
Row number: 22 of 9367
Row number: 23 of 9367
Row number: 24 of 9367
Row number: 25 of 9367
Row number: 26 of 9367
Row number: 27 of 9367
Row number: 28 of 9367
Row number: 29 of 9367
Row number: 30 of 9367
Row number: 31 of 9367
Row number: 32 of 9367
Row number: 33 of 9367
Row number: 34 of 9367
Row number: 35 of 9367
Row number: 36 of 9367
Row number: 37 of 9367
Row number: 38 of 9367
Row number: 39 of 9367
Row number: 40 of 9367
Row number: 41 of 9367
Row number: 42 of 9367
Row number: 43 of 936

Row number: 348 of 9367
Row number: 349 of 9367
Row number: 350 of 9367
Row number: 351 of 9367
Row number: 352 of 9367
Row number: 353 of 9367
Row number: 354 of 9367
Row number: 355 of 9367
Row number: 356 of 9367
Row number: 357 of 9367
Row number: 358 of 9367
Row number: 359 of 9367
Row number: 360 of 9367
Row number: 361 of 9367
Row number: 362 of 9367
Row number: 363 of 9367
Row number: 364 of 9367
Row number: 365 of 9367
Row number: 366 of 9367
Row number: 367 of 9367
Row number: 368 of 9367
Row number: 369 of 9367
Row number: 370 of 9367
Row number: 371 of 9367
Row number: 372 of 9367
Row number: 373 of 9367
Row number: 374 of 9367
Row number: 375 of 9367
Row number: 376 of 9367
Row number: 377 of 9367
Row number: 378 of 9367
Row number: 379 of 9367
Row number: 380 of 9367
Row number: 381 of 9367
Row number: 382 of 9367
Row number: 383 of 9367
Row number: 384 of 9367
Row number: 385 of 9367
Row number: 386 of 9367
Row number: 387 of 9367
Row number: 388 of 9367
Row number: 389 

Row number: 692 of 9367
Row number: 693 of 9367
Row number: 694 of 9367
Row number: 695 of 9367
Row number: 696 of 9367
Row number: 697 of 9367
Row number: 698 of 9367
Row number: 699 of 9367
Row number: 700 of 9367
Row number: 701 of 9367
Row number: 702 of 9367
Row number: 703 of 9367
Row number: 704 of 9367
Row number: 705 of 9367
Row number: 706 of 9367
Row number: 707 of 9367
Row number: 708 of 9367
Row number: 709 of 9367
Row number: 710 of 9367
Row number: 711 of 9367
Row number: 712 of 9367
Row number: 713 of 9367
Row number: 714 of 9367
Row number: 715 of 9367
Row number: 716 of 9367
Row number: 717 of 9367
Row number: 718 of 9367
Row number: 719 of 9367
Row number: 720 of 9367
Row number: 721 of 9367
Row number: 722 of 9367
Row number: 723 of 9367
Row number: 724 of 9367
Row number: 725 of 9367
Row number: 726 of 9367
Row number: 727 of 9367
Row number: 728 of 9367
Row number: 729 of 9367
Row number: 730 of 9367
Row number: 731 of 9367
Row number: 732 of 9367
Row number: 733 

Row number: 1036 of 9367
Row number: 1037 of 9367
Row number: 1038 of 9367
Row number: 1039 of 9367
Row number: 1040 of 9367
Row number: 1041 of 9367
Row number: 1042 of 9367
Row number: 1043 of 9367
Row number: 1044 of 9367
Row number: 1045 of 9367
Row number: 1046 of 9367
Row number: 1047 of 9367
Row number: 1048 of 9367
Row number: 1049 of 9367
Row number: 1050 of 9367
Row number: 1051 of 9367
Row number: 1052 of 9367
Row number: 1053 of 9367
Row number: 1054 of 9367
Row number: 1055 of 9367
Row number: 1056 of 9367
Row number: 1057 of 9367
Row number: 1058 of 9367
Row number: 1059 of 9367
Row number: 1060 of 9367
Row number: 1061 of 9367
Row number: 1062 of 9367
Row number: 1063 of 9367
Row number: 1064 of 9367
Row number: 1065 of 9367
Row number: 1066 of 9367
Row number: 1067 of 9367
Row number: 1068 of 9367
Row number: 1069 of 9367
Row number: 1070 of 9367
Row number: 1071 of 9367
Row number: 1072 of 9367
Row number: 1073 of 9367
Row number: 1074 of 9367
Row number: 1075 of 9367


Row number: 1365 of 9367
Row number: 1366 of 9367
Row number: 1367 of 9367
Row number: 1368 of 9367
Row number: 1369 of 9367
Row number: 1370 of 9367
Row number: 1371 of 9367
Row number: 1372 of 9367
Row number: 1373 of 9367
Row number: 1374 of 9367
Row number: 1375 of 9367
Row number: 1376 of 9367
Row number: 1377 of 9367
Row number: 1378 of 9367
Row number: 1379 of 9367
Row number: 1380 of 9367
Row number: 1381 of 9367
Row number: 1382 of 9367
Row number: 1383 of 9367
Row number: 1384 of 9367
Row number: 1385 of 9367
Row number: 1386 of 9367
Row number: 1387 of 9367
Row number: 1388 of 9367
Row number: 1389 of 9367
Row number: 1390 of 9367
Row number: 1391 of 9367
Row number: 1392 of 9367
Row number: 1393 of 9367
Row number: 1394 of 9367
Row number: 1395 of 9367
Row number: 1396 of 9367
Row number: 1397 of 9367
Row number: 1398 of 9367
Row number: 1399 of 9367
Row number: 1400 of 9367
Row number: 1401 of 9367
Row number: 1402 of 9367
Row number: 1403 of 9367
Row number: 1404 of 9367


...

Row number: 7952 of 9367
Row number: 7953 of 9367
Row number: 7954 of 9367
Row number: 7955 of 9367
Row number: 7956 of 9367
Row number: 7957 of 9367
Row number: 7958 of 9367
Row number: 7959 of 9367
Row number: 7960 of 9367
Row number: 7961 of 9367
Row number: 7962 of 9367
Row number: 7963 of 9367
Row number: 7964 of 9367
Row number: 7965 of 9367
Row number: 7966 of 9367
Row number: 7967 of 9367
Row number: 7968 of 9367
Row number: 7969 of 9367
Row number: 7970 of 9367
Row number: 7971 of 9367
Row number: 7972 of 9367
Row number: 7973 of 9367
Row number: 7974 of 9367
Row number: 7975 of 9367
Row number: 7976 of 9367
Row number: 7977 of 9367
Row number: 7978 of 9367
Row number: 7979 of 9367
Row number: 7980 of 9367
Row number: 7981 of 9367
Row number: 7982 of 9367
Row number: 7983 of 9367
Row number: 7984 of 9367
Row number: 7985 of 9367
Row number: 7986 of 9367
Row number: 7987 of 9367
Row number: 7988 of 9367
Row number: 7989 of 9367
Row number: 7990 of 9367
Row number: 7991 of 9367


Row number: 8281 of 9367
Row number: 8282 of 9367
Row number: 8283 of 9367
Row number: 8284 of 9367
Row number: 8285 of 9367
Row number: 8286 of 9367
Row number: 8287 of 9367
Row number: 8288 of 9367
Row number: 8289 of 9367
Row number: 8290 of 9367
Row number: 8291 of 9367
Row number: 8292 of 9367
Row number: 8293 of 9367
Row number: 8294 of 9367
Row number: 8295 of 9367
Row number: 8296 of 9367
Row number: 8297 of 9367
Row number: 8298 of 9367
Row number: 8299 of 9367
Row number: 8300 of 9367
Row number: 8301 of 9367
Row number: 8302 of 9367
Row number: 8303 of 9367
Row number: 8304 of 9367
Row number: 8305 of 9367
Row number: 8306 of 9367
Row number: 8307 of 9367
Row number: 8308 of 9367
Row number: 8309 of 9367
Row number: 8310 of 9367
Row number: 8311 of 9367
Row number: 8312 of 9367
Row number: 8313 of 9367
Row number: 8314 of 9367
Row number: 8315 of 9367
Row number: 8316 of 9367
Row number: 8317 of 9367
Row number: 8318 of 9367
Row number: 8319 of 9367
Row number: 8320 of 9367


Row number: 8609 of 9367
Row number: 8610 of 9367
Row number: 8611 of 9367
Row number: 8612 of 9367
Row number: 8613 of 9367
Row number: 8614 of 9367
Row number: 8615 of 9367
Row number: 8616 of 9367
Row number: 8617 of 9367
Row number: 8618 of 9367
Row number: 8619 of 9367
Row number: 8620 of 9367
Row number: 8621 of 9367
Row number: 8622 of 9367
Row number: 8623 of 9367
Row number: 8624 of 9367
Row number: 8625 of 9367
Row number: 8626 of 9367
Row number: 8627 of 9367
Row number: 8628 of 9367
Row number: 8629 of 9367
Row number: 8630 of 9367
Row number: 8631 of 9367
Row number: 8632 of 9367
Row number: 8633 of 9367
Row number: 8634 of 9367
Row number: 8635 of 9367
Row number: 8636 of 9367
Row number: 8637 of 9367
Row number: 8638 of 9367
Row number: 8639 of 9367
Row number: 8640 of 9367
Row number: 8641 of 9367
Row number: 8642 of 9367
Row number: 8643 of 9367
Row number: 8644 of 9367
Row number: 8645 of 9367
Row number: 8646 of 9367
Row number: 8647 of 9367
Row number: 8648 of 9367


Row number: 8938 of 9367
Row number: 8939 of 9367
Row number: 8940 of 9367
Row number: 8941 of 9367
Row number: 8942 of 9367
Row number: 8943 of 9367
Row number: 8944 of 9367
Row number: 8945 of 9367
Row number: 8946 of 9367
Row number: 8947 of 9367
Row number: 8948 of 9367
Row number: 8949 of 9367
Row number: 8950 of 9367
Row number: 8951 of 9367
Row number: 8952 of 9367
Row number: 8953 of 9367
Row number: 8954 of 9367
Row number: 8955 of 9367
Row number: 8956 of 9367
Row number: 8957 of 9367
Row number: 8958 of 9367
Row number: 8959 of 9367
Row number: 8960 of 9367
Row number: 8961 of 9367
Row number: 8962 of 9367
Row number: 8963 of 9367
Row number: 8964 of 9367
Row number: 8965 of 9367
Row number: 8966 of 9367
Row number: 8967 of 9367
Row number: 8968 of 9367
Row number: 8969 of 9367
Row number: 8970 of 9367
Row number: 8971 of 9367
Row number: 8972 of 9367
Row number: 8973 of 9367
Row number: 8974 of 9367
Row number: 8975 of 9367
Row number: 8976 of 9367
Row number: 8977 of 9367


Row number: 9269 of 9367
Row number: 9270 of 9367
Row number: 9271 of 9367
Row number: 9272 of 9367
Row number: 9273 of 9367
Row number: 9274 of 9367
Row number: 9275 of 9367
Row number: 9276 of 9367
Row number: 9277 of 9367
Row number: 9278 of 9367
Row number: 9279 of 9367
Row number: 9280 of 9367
Row number: 9281 of 9367
Row number: 9282 of 9367
Row number: 9283 of 9367
Row number: 9284 of 9367
Row number: 9285 of 9367
Row number: 9286 of 9367
Row number: 9287 of 9367
Row number: 9288 of 9367
Row number: 9289 of 9367
Row number: 9290 of 9367
Row number: 9291 of 9367
Row number: 9292 of 9367
Row number: 9293 of 9367
Row number: 9294 of 9367
Row number: 9295 of 9367
Row number: 9296 of 9367
Row number: 9297 of 9367
Row number: 9298 of 9367
Row number: 9299 of 9367
Row number: 9300 of 9367
Row number: 9301 of 9367
Row number: 9302 of 9367
Row number: 9303 of 9367
Row number: 9304 of 9367
Row number: 9305 of 9367
Row number: 9306 of 9367
Row number: 9307 of 9367
Row number: 9308 of 9367
